## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-24-19-13-29 +0000,nyt,"Racing to Catch Up With Nvidia, AMD Signs Chip...",https://www.nytimes.com/2026/02/24/business/me...
1,2026-02-24-19-12-04 +0000,nyt,Home Depot Says Homeowners Are Weary From Econ...,https://www.nytimes.com/2026/02/24/business/ho...
2,2026-02-24-19-12-00 +0000,wsj,"Silver Gains, Gold Falls Ahead of Trump State ...",https://www.wsj.com/finance/commodities-future...
3,2026-02-24-19-09-54 +0000,bbc,Everything you need to know about the new scho...,https://www.bbc.com/news/videos/c5ygpz9yr3no?a...
4,2026-02-24-19-09-49 +0000,nyt,What Travelers Need to Know as Cartel Violence...,https://www.nytimes.com/2026/02/23/travel/cart...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2458/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,70
21,union,27
25,new,27
33,mexico,24
320,war,23


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
292,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,199
200,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...,188
343,2026-02-23-22-42-00 +0000,wsj,The Trump administration is considering new na...,https://www.wsj.com/politics/policy/trump-cons...,160
234,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,160
92,2026-02-24-16-59-59 +0000,nypost,Trump weighs new batch of national security ta...,https://nypost.com/2026/02/24/business/trump-w...,148


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
292,199,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...
316,97,2026-02-24-00-00-00 +0000,wsj,Mexico Races to Prevent Cartel War After the K...,https://www.wsj.com/world/americas/mexico-race...
211,67,2026-02-24-11-26-35 +0000,nypost,Ex-British Ambassador to US Peter Mandelson re...,https://nypost.com/2026/02/24/world-news/ex-br...
200,67,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...
355,57,2026-02-23-22-17-00 +0000,wsj,Rep. Tony Gonzales of Texas faced calls from R...,https://www.wsj.com/politics/policy/tony-gonza...
66,47,2026-02-24-17-39-21 +0000,nypost,Even more snow may hit Northeast cities alread...,https://nypost.com/2026/02/24/us-news/even-mor...
248,46,2026-02-24-09-00-00 +0000,cbc,'Putin will not stop': Zelenskyy warns world o...,https://www.cbc.ca/news/world/zelenskyy-trump-...
13,44,2026-02-24-18-53-54 +0000,bbc,Mandelson's lawyers say he was arrested over '...,https://www.bbc.com/news/articles/c05vey47yddo...
332,40,2026-02-23-23-09-36 +0000,nypost,El Mencho killing marks new war against cartel...,https://nypost.com/2026/02/23/world-news/el-me...
39,39,2026-02-24-18-16-30 +0000,nypost,CUNY professor who made ‘blatantly racist’ rem...,https://nypost.com/2026/02/24/us-news/cuny-pro...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
